<h1><center>Übungszettel 6</center></h1>
<h2><center>Inhaltsbasierte Bild- und Videoanalyse - SoSe 19</center></h2>
<h3><center>Abgabe: Di. 02.07.2019 - 12:00 Uhr</center></h3>

# CNNs in Tensorflow (2+3+4+3+3 Punkte)

In dieser Aufgabe soll ein neuronales Netz zur Klassifizierung von Bildern basierend
auf dem CIFAR-10-Datensatz (https://www.cs.toronto.edu/~kriz/cifar.html)realisiert werden. Dazu soll die Tensorflow API Keras (https://keras.io/) verwendet werden.

Der Datensatz besteht aus einem Trainings- und einem Testdatensatz. 

In [1]:
from __future__ import print_function
# -*- coding: utf-8 -*-
%matplotlib inline

import numpy as np

from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

np.random.seed(1000)

Using TensorFlow backend.


### a)  Preprocessing (2 Punkte)
Laden Sie zunächst die Trainings- und Testdaten des CIFAR-10 Dataset herunter. Nutzen Sie dafür `keras.datasets.cifar`. Berechnen Sie dann das Meanbild und ziehen es von den Bildern in den Datasets ab. Teilen Sie jeden Input-Wert durch 128.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [4]:
import matplotlib.pyplot as plt

x_train_mean_image = np.mean(x_train, axis=0, dtype='uint8')
x_test_mean_image = np.mean(x_test, axis=0, dtype='uint8')

x_train -= x_train_mean_image
x_train = x_train / 128

x_test -= x_test_mean_image
x_test = x_test / 128

y_train_mean_image = np.mean(y_train, axis=0, dtype='uint8')
y_test_mean_image = np.mean(y_test, axis=0, dtype='uint8')

y_train -= y_train_mean_image
y_train = y_train / 128

y_test -= y_test_mean_image
y_test = y_test / 128

### b)  Netzwerkarchitektur entwerfen (3 Punkte)
Erzeugen Sie nun eine Netzwerkarchitektur die für Bilder aus dem CIFAR-10 Dataset Wahrscheinlichkeiten für 10 Klassen vorhersagt. Bauen Sie das Modell nach folgenden folgende Vorgaben:
* Convolutional Layer: Aktivierung: ReLU, Kernel: 3x3, Padding: 1
* Pooling-Size: 2x2
* Struktur:
    - Input: 32x32x3
    
    - Convolution (32 Filter)
    - Batch Normalization
    - Convolution (64 Filter)
    - Batch Normalization
    - Max-Pooling
    
    - Convolution (128 Filter)
    - Batch Normalization
    - Max-Pooling
    
    - Convolution (128 Filter)
    - Batch Normalization
    - Max-Pooling
    
    - Fully-Connected (256 Neuronen)
    - Dropout (0.5)
    - Batch Normalization
    - Fully-Connected, 10 Outputs, Softmax-Aktivierung
    
Geben Sie auch die Struktur des Modells mit `model.summary()` aus.

In [5]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

W0630 23:50:09.534320  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0630 23:50:09.724262  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0630 23:50:09.828231  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0630 23:50:09.955193  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0630 23:50:09.956193  9864 deprecation_wrapper.py:119] From e:\pythons\pyth

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 12, 128)       512       
__________

### c)  Training (4 Punkte)
Trainieren Sie nun das Netz mit Stochastic Gradient Descent auf dem Trainingsdatensatz.
Nutzen Sie dafür das Nesterov Verfahren und setzen Sie sinnvolle Werte für Batchsize und Startlernrate.

Verwenden Sie Callbacks für TensorBoard (mit Bildern und Graph) und ReduceLROnPlateau (mögliche Werte: factor=0.1, patience=3, min_delta=0.001, min_lr=0.0001). 
Trainieren Sie das Modell für mindestens 10 Epochen. Nach etwa 50-100 Epochen sollten Sie gute Ergebnisse erhalten.
Binden Sie ein Bild der Losskurve in dieses Notebook ein.
Vergessen Sie nicht, das Modell zu speichern. Fügen Sie das gespeicherte (finale) Modell ihrer Abgabe hinzu.

In [6]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001, min_delta=0.001, verbose=1)

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test), 
          callbacks=[tbCallBack, reduce_lr])

model.save('CNN.h5')

W0630 23:50:10.654095  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Train on 50000 samples, validate on 10000 samples


W0630 23:50:26.495965  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0630 23:50:26.496964  9864 deprecation_wrapper.py:119] From e:\pythons\python37\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/10
50000/50000 [==============================] - 369s 7ms/step - loss: 0.0262 - val_loss: 0.0024
Epoch 2/10
50000/50000 [==============================] - 355s 7ms/step - loss: 0.0109 - val_loss: 0.0129
Epoch 3/10
50000/50000 [==============================] - 348s 7ms/step - loss: 0.0091 - val_loss: 0.0047
Epoch 4/10
50000/50000 [==============================] - 349s 7ms/step - loss: 0.0105 - val_loss: 4.2639e-04
Epoch 5/10
50000/50000 [==============================] - 349s 7ms/step - loss: 0.0018 - val_loss: 1.0827e-04
Epoch 6/10
50000/50000 [==============================] - 356s 7ms/step - loss: 2.5227e-04 - val_loss: 1.9713e-05
Epoch 7/10
50000/50000 [==============================] - 354s 7ms/step - loss: 1.2373e-04 - val_loss: 2.4041e-06

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
Epoch 8/10
50000/50000 [==============================] - 355s 7ms/step - loss: 7.9335e-05 - val_loss: 3.8525e-06
Epoch 9/10
50000/50000 [===============

### c)  Testen (3 Punkte)
Laden Sie das zuvor trainierte Modell. Evaluieren Sie nun das Modell auf den Testdaten.  

In [ ]:
from keras.models import load_model

model = load_model('CNN.h5')

### d)  Anwenden (3 Punkte)
Laden Sie das zuvor trainierte Modell. Wenden Sie das Modell auf 10 zufällig ausgewählte Bilder aus dem Testdatensatz an. Visualisieren Sie die Ergebnisse. Geben Sie dazu das Bild aus und die vorhergesagte Wahrscheinlichkeite für jede der 10 Klassen.